# youtube_transcript_api

In [ ]:
import pandas as pd
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
# Recarrega funções definidas a cada 2 segundos;
# evita ter de reiniciar o kernel o tempo todo
# ao desenvolver ou modificar alguma função
%load_ext autoreload
%autoreload 2

___

# Download de legendas de vários vídeos de vez

Aqui definimos uma classe `YouTubeWrapper` contendo apenas duas funções ou [métodos estáticos](https://docs.python.org/pt-br/3/library/functions.html?highlight=staticmethod#staticmethod) (`@staticmethod`):

> * `get_subtitles`: baixa a legenda de (um ou múltiplos) vídeos do YouTube

> * `get_video_ids`: lê um arquivo contendo IDs de vídeos do YouTube (um por linha, ignorando o cabeçário, i.e., a primeira linha)

In [ ]:
class YouTubeWrapper():
    '''
    Get subtitles using youtube_transcript_api.
    '''
    @staticmethod
    def get_subtitles(video_ids, languages=["pt", "pt-BR", "en"]):
        '''
        video_ids: list of YouTube IDs to get subtitles
        '''
        transcripts = {}
        
        if type(video_ids) == str:
            # Cria uma lista separada por vírgulas se necessário
            video_ids = video_ids.split(",")
        
        for video_id in video_ids:
            try:
                # Tenta baixar a legenda (transcript)
                transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)
            except:
                # Avisa do erro, mas não termina o loop
                print("Erro em baixar a legenda do vídeo: https://www.youtube.com/watch?v=%s." % video_id)
            else:
                # Carrega a legenda (transcript) como dataframe (df),
                # substitui quebras de linha ("\n") por espaços (" ")
                # e salva como um arquivo em formato CSV
                df = pd.DataFrame(transcript).applymap(lambda x: x.replace("\n", " ") if type(x) == str else x)
                df.to_csv("legenda_%s.csv" % video_id, index=False)
                transcripts[video_id] = df
        
        print("Got %s subtitles." % len(transcripts))
        return transcripts

    @staticmethod
    def get_video_ids(file_name, header=True):
        '''
        file_name: path to file to load YouTube IDs from
        '''
        with open(file_name, "r") as f:
            video_ids = [
                line.split("/")[-1].strip()
                for i, line in enumerate(f.readlines())
                if (i > 0 or header is False)
            ]
        print("Read %s lines." % len(video_ids))
        return video_ids

___

### Inicializa a classe e define opções

Inicializamos a classe `YouTubeWrapper` definida acima como a variável `yt` e definimos uma lista de IDs de vídeos (`video_ids`).

In [ ]:
yt = YouTubeWrapper()

In [ ]:
video_ids = ["dSu5sXmsur4", "BmX86O2ozdo", "Bpfw47x5a90"]

##### Opcional: lê os IDs de vídeos de um arquivo

Lê um arquivo contendo **IDs ou URLs** de vídeos no YouTube (um por linha). Por padrão, ignora a primeira linha de cabeçário (`header=True`).

In [ ]:
video_ids = yt.get_video_ids("video_ids.txt", header=True)

### Download da legenda de um ou mais vídeos

Para uma lista contendo um ou mais IDs de vídeos do YouTube (`video_ids`), retorna um dicionário (`transcripts`) contendo os resultados.

In [ ]:
transcripts = yt.get_subtitles(video_ids)

#### Lista de legendas obtidas

Retorna uma lista de chaves (IDs) do dicionário retornado (`transcripts`).

In [ ]:
transcripts.keys()

#### Visualizar dados de uma única legenda

Retorna o DataFrame de uma única legenda obtida (por padrão, a primeira retornada).

In [ ]:
video_id = list(transcripts.keys())[0]

transcripts[video_id]

#### Transforma em um data frame

Concatena **todas** as legendas retornadas em um único DataFrame.

In [ ]:
df = pd.concat(transcripts.values())
df

#### Extra: contagem das palavras mais ocorrentes

Retorna as cinquenta primeiras palavras mais ocorrentes no texto, excluindo-se as palavras com menos de quatro caracterers.

In [ ]:
df["text"] \
.apply(
    # dado um texto (x), realiza as operações:
    # converte para minúsculas (lower)
    # substitui quebras de linha por espaços (replace)
    # transforma em uma lista separada por espaços (split)
    # retorna apenas as palavras que tem 4 ou mais caracteres
    lambda x: [word for word in x.lower().replace("\n", " ").split() if len(word) > 4]
) \
.explode() \
.value_counts()[:50] \
.plot(kind="bar", figsize=(15,6))

___

## Outras funções

### Lista de legendas disponíveis de um vídeo

Retorna uma lista de legendas disponíveis (`transcript_list`), geradas manualmente (`manual`) ou automaticamente (`automatic`) pelo YouTube.

In [ ]:
video_id = "dSu5sXmsur4"
languages = ["pt", "pt-BR", "en"]

transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

# retorna apenas legendas criadas manualmente
manual = transcript_list.find_manually_created_transcript(languages)

# retorna apenas legendas criadas manualmente
automatic = transcript_list.find_generated_transcript(languages)

print("Legendas criadas manualmente:\n%s\n\nLegendas criadas automaticamente:\n%s" % (manual, automatic) )

### Baixar a legenda de um vídeo do YouTube

Retorna a legenda (`transcript`) de um vídeo no YouTube baseado em seu ID (`video_id`) e em uma lista de idiomas (`languages`).

In [ ]:
video_id = "dSu5sXmsur4"
languages = ["pt", "pt-BR", "en"]

In [ ]:
transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)

#### Inspecionando o objeto

A legenda é retornada no formato de uma lista (`list`) de dicionários (`dict`).

In [ ]:
type(transcript) # tipo da variável

In [ ]:
len(transcript) # número de elementos na lista

In [ ]:
type(transcript[0]) # tipo do primeiro objeto/elemento na lista

In [ ]:
transcript[0].keys() # chaves do dicionário

#### Salvar legenda em um arquivo CSV

O [Pandas](https://pandas.pydata.org/) automaticamente identifica uma lista de dicionários e o converte para um data frame (`df`). Cada chave se transforma em uma coluna.

In [ ]:
df = pd.DataFrame(transcript)
df

In [ ]:
df.to_csv("legenda_%s.csv" % video_id, index=False)

#### Salvar o arquivo manualmente

Salva a legenda para um arquivo **legenda.txt** aberto (`with open`) para escrita (`w` ou `write`) e definidido como **f** (`as f`).

In [ ]:
with open("legenda.txt", "w") as f:
    f.write("text\n")
    for t in transcript:
        f.write(t["text"] + "\n")

#### Alternativa: one-liner usando list comprehension

Salva a legenda em uma linha apenas usando **[list comprehension](https://docs.python.org/pt-br/3/tutorial/datastructures.html#list-comprehensions)** e aderindo às convenções para leitura/escrita de arquivos com `with`.

In [ ]:
with open("legenda.txt", "w") as f: f.write("text\n" + "\n".join([item["text"]+"\n" for item in transcript]))